https://www.data.go.kr/data/15058747/openapi.do

https://www.code.go.kr/stdcode/regCodeL.do

https://shiningyouandme.tistory.com/11

In [20]:
filename = 'ServiceKey 국토교통부 아파트매매 실거래.txt'
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(filename)
#print("Our API key is: %s" % (api_key))

from urllib.parse import urlencode, quote_plus
import requests
import pandas as pd
from bs4 import BeautifulSoup
import bs4

url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent?"
service_key = 


base_date = ["202007","202008","202009"]


In [23]:

import requests

url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade'
params ={'serviceKey' : api_key, 'LAWD_CD' : '11110', 'DEAL_YMD' : '20201212' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>'


In [15]:
import bs4 as bs
import urllib.request
import pandas as pd 

soup = bs.BeautifulSoup(response,'xml')


TypeError: object of type 'Response' has no len()

for i in range(len(base_date)):

    gu_code = '11545' ##구 단위로 데이터를 확보하는 것. ex)11545 = 금천구
    payload = "serviceKey=" + service_key + "&"+"LAWD_CD=" + gu_code + "&"+"DEAL_YMD=" + base_date[i]+ "&" 

    res = requests.get(url + payload).text
    xmlobj = bs4.BeautifulSoup(res, 'lxml-xml')
    rows = xmlobj.findAll('item')

    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        columnsLen = len(columns)
        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].nmame)
            eachColumn = columns[j].text
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    
result = pd.DataFrame(rowList, columns=nameList)

result

http://www.code.go.kr/stdcode/regCodeL.do